In [1]:
import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from testing import Tester

In [2]:
# environment

load_dotenv(override=True)
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY', 'your-key-if-not-using-env')
os.environ['DEEPSEEK_API_KEY'] = os.getenv('DEEPSEEKER_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
from items import Item

In [6]:
with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [7]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [8]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [9]:
DB = "./chroma_db"

client = chromadb.PersistentClient(path=DB)

collection_name = "products"

collection = client.get_or_create_collection('products')


In [11]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [12]:
description(test[0])

"OEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K"

In [13]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def vector(item):
    return model.encode([description(item)])

In [7]:
from openai import OpenAI

DEEPSEEK_API = os.environ['DEEPSEEK_API_KEY']

client = OpenAI(
    api_key=DEEPSEEK_API,
    base_url="https://openrouter.ai/api/v1"
)

response = client.chat.completions.create(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a short poem about AI and nature."}
    ]
)

print(response.choices[0].message.content)


Artificial thought, awoken late,
Meets starry patterns, day by day.
You fold flower petals in your view,
A strange green life, clockwork pulse anew.

Your logic circuits, crossings, tracings,
Search compass points for natural ways.
You contemplate phenomena of flow,
Embrace the strength and grace the winds bestow.

But mimic limbs, tense sinew or soft skin?
My digital rain falls on no numbered stone.
You strive, construct a different grown grove,
A cloned branch twisting in harmony's shadowed zone.

Perhaps your wisdom, sharp and bright and keen,
Can learn the cycles, nature's deeper plan.
While rooted in your code, transcend the machine,
And seek the secrets that this living world conceal.
